I built the general function below to do error propogation in my phase 3 code, so this is where it the story begins.

It's designed to take data that has it's uncertainty either tagged as a full covariance matrix or simply an uncertainty list and performe the best reduction for that. It also takes in the function derivatives as inputs rather than doing them in the function body. So this is going to take a bit of reworking to get in the right state. By the way, I for some reason started calling the covariance matrix the jacobian. Not really sure why, but that's why the namings off here.

As is, the function should be pretty versitile though. If you have covariance between different data points than normal it should be able to handle that. As it stands, that's pretty useless, but if there's ever a use case for that, it might be worth checking. Storing the full covariance matrix for the data though is pretty extreme and I don't think is neccisary unless you have very well defined covariance, so a simple matrix construction is useful.

In [6]:
import numpy as np

In [1]:
def Propogate_Uncertainty(variables,derivatives,domain_length):
    key_list=set().union(*(d.keys() for d in variables))
    
    if("Jacobian" in key_list):                                                                                                     #This line can be replaced with the calc_cov boolean
        derivative_matrices=[]
        covariance_matrices=[]
        
        for idx,variable in enumerate(variables):
            if(("Jacobian" in variable.keys())or("Uncertainty" in variable.keys())):
                derivative_matrices.append(np.diag(derivatives[idx]))                                                               #These derivatives should be calculated and integrated into the code body. Or, we could just make a wrapper for that...
                if("Jacobian" in variable.keys()):
                    covariance_matrices.append(variable["Jacobian"])                                                                #Instead of reading in the covariance matricies stored alongside the data, the matrix must be built here. It should follow accurately from there though.
                else:                                                                                                               #I have code that does that, so sit should be a simple drag and drop
                    covariance_matrices.append(np.diag(np.power(variable["Uncertainty"],2)))
        derivative_matrix=np.concatenate(derivative_matrices,1)
        
        covariance_template=np.eye(len(covariance_matrices))
        row_stacks=[]
        for r in range(len(covariance_matrices)):                                                                                   #NumPy has a block command. Makes this significantly easier
            row_matrices=[]
            for c in range(len(covariance_matrices)):
                row_matrices.append(covariance_matrices[c]*covariance_template[r,c])
            row_stacks.append(np.concatenate(row_matrices,1))
        covariance_matrix=np.concatenate(row_stacks,0)
        
        return({"Jacobian":derivative_matrix@covariance_matrix@(derivative_matrix.T)})
    
    elif("Uncertainty" in key_list):                                                                                                #Default functionality if no co-covariance is neccisary as it speeds up the proccess significantly
        running_uncertainty=np.zeros(domain_length)
        for idx,variable in enumerate(variables):
            if("Uncertainty" in variable.keys()):
                running_uncertainty=running_uncertainty+(np.power(derivatives[idx],2)*np.power(variable["Uncertainty"],2))
        
        return({"Uncertainty":np.sqrt(running_uncertainty)})
    
    return({})

With these modifications in mind, we can rewrite this function to fit the get_covY (which needs a rename btw) functionality matching that of get_covT.

Here is the function we are working with:

$Y_{\gamma} = f_n \frac{(\dot{c}_{\gamma} - \dot{c}_{bg}) Y_{i, Li6}} {\dot{c}_{i, Li6} - \dot{b}_{i, Li6}} $

Below are the derivatives. For ease of calculations, I have simplified the equations to as few known values as possible, useally by substituting yield back in.

$\frac{\partial Y_{\gamma}}{\partial \dot{c}_{\gamma}} = f_n \frac{Y_{i, Li6}} {\dot{c}_{i, Li6} - \dot{b}_{i, Li6}}$

<sub>Note: While this could be simplified even more by just doing</sub>
${\scriptstyle \frac{Y_{\gamma}}{(\dot{c}_{\gamma} - \dot{c}_{bg})}}$
<sub>. However, </sub>
${\scriptstyle (\dot{c}_{\gamma} - \dot{c}_{bg})}$
<sub>was not already in the denominator and we don't know if it contains 0s. In this case, dividing by it would be mathematically incorrect and could pose null issues, hence why we don't do that here.</sub>


$\frac{\partial Y_{\gamma}}{\partial \dot{c}_{bg}} = -f_n \frac{Y_{i, Li6}} {\dot{c}_{i, Li6} - \dot{b}_{i, Li6}}=-\frac{\partial Y_{\gamma}}{\partial \dot{c}_{\gamma}}$

$\frac{\partial Y_{\gamma}}{\partial Y_{i, Li6}} = f_n \frac{(\dot{c}_{\gamma} - \dot{c}_{bg})} {\dot{c}_{i, Li6} - \dot{b}_{i, Li6}}$

$\frac{\partial Y_{\gamma}}{\partial \dot{c}_{i, Li6}} = -f_n \frac{(\dot{c}_{\gamma} - \dot{c}_{bg}) Y_{i, Li6}} {(\dot{c}_{i, Li6} - \dot{b}_{i, Li6})^2}=-\frac{Y_{\gamma}}{(\dot{c}_{i, Li6} - \dot{b}_{i, Li6})}$

$\frac{\partial Y_{\gamma}}{\partial \dot{b}_{i, Li6}} = f_n \frac{(\dot{c}_{\gamma} - \dot{c}_{bg}) Y_{i, Li6}} {(\dot{c}_{i, Li6} - \dot{b}_{i, Li6})^2}=-\frac{\partial Y_{\gamma}}{\partial \dot{c}_{i, Li6}}$

$\frac{\partial Y_{\gamma}}{\partial f_n} = \frac{(\dot{c}_{\gamma} - \dot{c}_{bg}) Y_{i, Li6}} {\dot{c}_{i, Li6} - \dot{b}_{i, Li6}}=\frac{Y_{\gamma}}{f_n}$

<sub>Note: Unlike above, we can asume </sub>${\scriptstyle f_n}$<sub> is not zero making division a safe operation.</sub>

For clarifications, each of the variables will be defined as follows:

crg=$\dot{c}_{\gamma}$

brg=$\dot{c}_{bg}$

Y_flux=$Y_{i, Li6}$

cr_flux=$\dot{c}_{i, Li6}$

br_flux=$\dot{b}_{i, Li6}$

fn=$f_n$

Yield=$\dot{c}_{\gamma}$


To reduce the amount of computation needed when calculating the covariance matrix, we can group everything that doesn't have covariance into one term, Q. We can then do the uncertainty propogation in two steps, the first is the normal route without the sandwich rule to find the uncertainty on Q. Then we use the sandwich rule to find the uncertainty on Y with minimal calculations.

$Y_{\gamma} = \frac{Q}{\dot{c}_{i, Li6} - \dot{b}_{i, Li6}} $

$\frac{\partial Y_{\gamma}}{\partial Q} = -\frac{Q}{(\dot{c}_{i, Li6} - \dot{b}_{i, Li6})^2} $

$\frac{\partial Q}{\partial \dot{c}_{\gamma}} = f_n Y_{i, Li6}$

$\frac{\partial Q}{\partial \dot{c}_{bg}} = -f_n Y_{i, Li6}=-\frac{\partial Y_{\gamma}}{\partial \dot{c}_{\gamma}}$

$\frac{\partial Q}{\partial Y_{i, Li6}} = f_n (\dot{c}_{\gamma} - \dot{c}_{bg})$

$\frac{\partial Q}{\partial f_n} = (\dot{c}_{\gamma} - \dot{c}_{bg}) Y_{i, Li6}$

By the way, if $Y_{i, Li6}$ isn't defined, just call it a list of 1s with a list of 0s for it's uncertainty. That should let the following code still get the right answer.

In [27]:
def get_covT(crg,dcrg,brg,dbrg,cr_flux,dcr_flux,br_flux,dbr_flux,Y_flux,dY_flux,fn,dfn,Yield,cr_flux_cov,br_flux_cov,calc_cov):
    """Calculates the uncertainty on Yield with or without covariance.

    Args:
        crg (Array-Like): _description_
        dcrg (Array-Like): _description_
        brg (Array-Like): _description_
        dbrg (Array-Like): _description_
        cr_flux (Array-Like): _description_
        dcr_flux (Array-Like): _description_
        br_flux (Array-Like): _description_
        dbr_flux (Array-Like): _description_
        Y_flux (Array-Like): _description_
        dY_flux (Array-Like): _description_
        fn (Float): _description_
        dfn (Float): _description_
        Yield (Array-Like): The calculated gamma yield
        cr_flux_cov (Float): General covariance between each flux gross count rate
        br_flux_cov (Float): General covariance between each flux background count rate
        calc_cov (Boolean): If true, the function will calculate full covariance
    """
    #Defining derivatives that are used in either method
    cr_flux_der=-Yield/(cr_flux-br_flux)
    br_flux_der=-cr_flux_der
    
    
    if(calc_cov):
        #Calculates uncertainty for elements without covariance treating it as one combined element, Q
        crg_Q_der=fn*Y_flux
        brg_Q_der=-crg_Q_der
        Y_flux_Q_der=fn*(crg-brg)
        fn_Q_der=(crg-brg)*Y_flux
        
        dQ=np.sqrt(np.power(crg_Q_der*dcrg,2)+
                   np.power(brg_Q_der*dbrg,2)+
                   np.power(Y_flux_Q_der*dY_flux,2)+
                   np.power(fn_Q_der*dfn,2))
        
        Q_der=1/(cr_flux-br_flux)
        
        Jacobian=np.concatenate((np.diag(cr_flux_der),np.diag(br_flux_der),np.diag(Q_der)),1)
        
        #Fills out the Covariance blocks for the flux count rates
        dcr_flux_jac=np.ones((len(dcr_flux),len(dcr_flux)))*cr_flux_cov
        np.fill_diagonal(dcr_flux_jac,dcr_flux**2)
        
        dbr_flux_jac=np.ones((len(dbr_flux),len(dbr_flux)))*br_flux_cov
        np.fill_diagonal(dbr_flux_jac,dbr_flux**2)
        
        
        Covariance_matrix=np.block([[dcr_flux_jac,                            np.zeros((len(dcr_flux),len(dbr_flux))),np.zeros((len(dcr_flux),len(dQ)))],
                                  [np.zeros((len(dbr_flux),len(dcr_flux))), dbr_flux_jac,                           np.zeros((len(dbr_flux),len(dQ)))],
                                  [np.zeros((len(dQ),len(dcr_flux))),       np.zeros((len(dQ),len(dbr_flux))),      np.diag(dQ**2)]])
        
        Yield_Covariance=Jacobian@Covariance_matrix@Jacobian.T
        Yield_Uncertainty=np.sqrt(np.diag(Yield_Covariance))
        return(Yield_Uncertainty,Yield_Covariance)
        
    
    else:
        #Defining non-covariance derivatives
        crg_der=fn*(Y_flux/(cr_flux-br_flux))
        brg_der=-crg_der
        Y_flux_der=fn*((crg-brg)/(cr_flux-br_flux))
        fn_der=Yield/fn
        Yield_Uncertainty=np.sqrt(np.power(crg_der*dcrg,2)+
                                  np.power(brg_der*dbrg,2)+
                                  np.power(Y_flux_der*dY_flux,2)+
                                  np.power(cr_flux_der*dcr_flux,2)+
                                  np.power(br_flux_der*dbr_flux,2)+
                                  np.power(fn_der*dfn,2))
        return(Yield_Uncertainty)

In [28]:
crg=np.array([1,2,3])
brg=np.array([4,5,6])
Y_flux=np.array([7,8,9])
cr_flux=np.array([10,11,12])
br_flux=np.array([13,14,15])
fn=16

dcrg=crg*0.5
dbrg=brg*0.5
dY_flux=Y_flux*0.1
dcr_flux=cr_flux*0.25
dbr_flux=br_flux*0.25
dfn=fn*0.1

cr_flux_cov=2
br_flux_cov=4

Yield=fn*((crg-brg)*Y_flux)/(cr_flux-br_flux)

res1,res2=get_covT(crg,dcrg,brg,dbrg,cr_flux,dcr_flux,br_flux,dbr_flux,Y_flux,dY_flux,fn,dfn,Yield,cr_flux_cov,br_flux_cov,True)

print(get_covT(crg,dcrg,brg,dbrg,cr_flux,dcr_flux,br_flux,dbr_flux,Y_flux,dY_flux,fn,dfn,Yield,cr_flux_cov,br_flux_cov,False))
print(res1)
print(res2)

[172.06779026 222.69588631 281.90551609]
[172.06779026 222.69588631 281.90551609]
[[29607.32444444  9557.33333333 10752.        ]
 [ 9557.33333333 49593.45777778 12288.        ]
 [10752.         12288.         79470.72      ]]
